# Test of whether LanguageModelLoader deliver batch where each row is a continuous set of tokens


Status is that the test works with

-MyLanguageModelLoader 

-fastai's LanguageModelLoader, however, throws an exception while indexing the jagged array


In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
from fastai.text import * 
from languagemodelloader import *

In [5]:
def getAllBatches(data,epochs=1):
    x=None
    for i in range(epochs):
        for xb,yb in data:
            d= xb.data.numpy()
            if x is None: 
                x = xb.data.numpy().copy()
            else:         
                x = np.concatenate((x, xb.data.numpy().copy()),axis=1)
            continue
    return x            

In [6]:
def generateContinuousNumber(bs,seq_len,iterations,minTokens):
    "The data consist of continuously increasing numbers i jagged array with randomly varying layout"
    jagged = []
    countTokens = 0
    totalTokens = bs*seq_len*iterations
    #print(f"totalTokens:{totalTokens}")
    rand_interval=20
    while countTokens < totalTokens:
        nb = totalTokens-countTokens if   totalTokens-countTokens <rand_interval\
                                     else minTokens+int(np.random.random() * rand_interval)
        jagged.append(np.arange(countTokens+1,countTokens+1+nb))
        countTokens = jagged[-1][-1]
    jagged = np.asarray(jagged)    
    return jagged, countTokens

def printJagged(jagged, countTokens):
    print(f"countTokens:{countTokens}")
    for j in jagged: print(j)

In [44]:
def test_datadirection( bs,seq_len,iterations,minTokens, backwards=False, nbInoutGenerations=10, log=False):
    for i in range(nbInoutGenerations):
        jagged,countTokens = generateContinuousNumber(bs,seq_len,iterations,minTokens)
        if log: 
            print("jagged array")
            printJagged(jagged, countTokens)
            
        trainIDS = validIDS = jagged
        db      = TextLMDataBunch.from_ids( ".", None, trainIDS, validIDS, bptt=seq_len, bs=bs)
        #data    = MyLanguageModelPreLoader(db.train_ds, bs, seq_len, backwards=backwards, shuffle=False, log=False)
        data    = LanguageModelPreLoader(db.train_ds, bs=bs, bptt=seq_len, backwards=backwards, shuffle=False)
        batches = getAllBatches(DataLoader(data, bs, shuffle=False))
        if log: 
            print("batchtokens")
            display(pd.DataFrame(data=batches))

        #The diff from one to the next column must be 1 for aligned mini-batches with forward indexing of the data
        #(forward is default for LanguageModelLoader ie.: backwards=False) 
        b_diff = batches[:,1:] - batches[:,0:-1]
        if log: 
            print("batch token diffs")
            display(pd.DataFrame(data=b_diff))
            
        diff_value = -1 if backwards else 1
        assert (b_diff.flatten()==diff_value).all()

# Test continouity of tokens in batches loaded forwards and backwards

In [45]:
#create test data so that we can control whether the LanguageModelLoader returns contigous tokens 
#The genrated data must be size so that the batches will not wrap aoround. 
bs         =  4
seq_len    =  8  #=bptt
iterations =  2
minTokens  =  1 #in a rag array

In [48]:
%time test_datadirection( bs, seq_len, iterations, minTokens, backwards=False, nbInoutGenerations=10, log=True)

jagged array
countTokens:64
[1 2 3 4 5]
[ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[21 22 23]
[24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]
[41 42 43 44 45 46 47 48 49 50 51 52 53 54 55]
[56 57 58 59 60 61 62 63 64]
batchtokens


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,2,3,4,5,6,7,8,1,2,3,4,5,6,7,8
1,17,18,19,20,21,22,23,24,17,18,19,20,21,22,23,24
2,33,34,35,36,37,38,39,40,33,34,35,36,37,38,39,40
3,49,50,51,52,53,54,55,56,49,50,51,52,53,54,55,56


batch token diffs


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,1,1,1,1,1,1,-7,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,-7,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,-7,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,-7,1,1,1,1,1,1,1


AssertionError: 

In [42]:
%time test_datadirection( bs, seq_len, iterations, minTokens, backwards=True, nbInoutGenerations=10000, log=False)

Wall time: 7.72 s
